In [27]:
import numpy as np 
import scipy as sp 
import matplotlib.pyplot as plt 
from sklearn.datasets import load_iris
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.discriminant_analysis import * 
from sklearn.pipeline import make_pipeline

iris=load_iris()
x=iris.data
y=iris.target
##repartition
xtrain,xtest,ytrain,ytest=train_test_split(x,y,train_size=0.7,random_state=5)
##standarisation
transformer=StandardScaler()
xtrain_trans2=transformer.fit_transform(xtrain)
xtest_trans2=transformer.transform(xtest)

##crossvalidaion
param_grid={'solver':['svd','eigen','lsqr']}
grid=GridSearchCV(LinearDiscriminantAnalysis(),param_grid,cv=5)
grid.fit(xtrain_trans2,ytrain)
print(grid.best_params_)
print(grid.best_score_)


##model
model=LinearDiscriminantAnalysis()
model.fit(xtrain_trans2,ytrain)
print("prediction 1 =>",model.predict(xtest_trans2))
print("score 1 =>",model.score(xtest_trans2,ytest))

##pipline :une chaine de transformation qui contient estimator et le transformer en meme temps
##pourqoui une pipline : plus simple a lire 
#plus simple a utiliser evite de faire des fuites de donnees

modele2=make_pipeline(PolynomialFeatures(),LinearDiscriminantAnalysis())
modele2.fit(xtrain,ytrain)
print("prediction 2 =>",modele2.predict(xtest))
print("score 2 =>",modele2.score(xtest,ytest))



### GridSearchCV with Pipeline ######################
param_grid2={
    'polynomialfeatures__degree':[1,2,3],
    'lineardiscriminantanalysis__solver':['svd','eigen','lsqr']
}
grid2=GridSearchCV(modele2,param_grid2,cv=5)
#param=model.get_params()
#print("parametre modele",param)
grid2.fit(xtrain,ytrain)
print("prediction 3=>",grid2.predict(xtest))
print("Score =>",grid2.score(xtest,ytest))
print(" Meilleure score =>",grid2.score(xtest,ytest))
print(" meilleures parametres =>",grid2.best_params_)
                      














{'solver': 'svd'}
0.980952380952381
prediction 1 => [1 2 2 0 2 1 0 2 0 1 1 1 2 2 0 0 2 2 0 0 1 2 0 1 1 2 1 1 1 2 0 1 1 0 1 0 0
 2 0 2 2 1 0 0 1]
score 1 => 0.9555555555555556
prediction 2 => [1 2 2 0 2 1 0 1 0 1 1 1 2 2 0 0 2 2 0 0 1 2 0 1 1 2 1 1 1 2 0 1 1 0 1 0 0
 2 0 2 2 1 0 0 1]
score 2 => 0.9777777777777777


c:\users\packard bell\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\packard bell\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\packard bell\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "c:\users\packard bell\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\discriminant_analysis.py", line 560, in fit
    covariance_estimator=self.covariance_estimator)
  File "c:\users\packard bell\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\discriminant_analy

prediction 3=> [1 2 2 0 2 1 0 2 0 1 1 1 2 2 0 0 2 2 0 0 1 2 0 1 1 2 1 1 1 2 0 1 1 0 1 0 0
 2 0 2 2 1 0 0 1]
Score => 0.9555555555555556
 Meilleure score => 0.9555555555555556
 meilleures parametres => {'lineardiscriminantanalysis__solver': 'svd', 'polynomialfeatures__degree': 1}


c:\users\packard bell\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [0.98095238 0.98095238 0.93333333        nan        nan        nan
 0.98095238 0.98095238 0.94285714]
  category=UserWarning


In [77]:
########### pipeline v2##################
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import * 
from sklearn.preprocessing import * 
from sklearn.discriminant_analysis import * 
from sklearn.model_selection import *
from sklearn.compose import *
from sklearn.impute import *
import numpy as np
import seaborn as sns

titanic=pd.read_excel("titanic.xls")
titanic.drop(['boat','body','home.dest'],axis=1,inplace=True)
titanic.dropna(axis=0,inplace=True)
print(titanic.head())
#print(titanic.head())
x= titanic[['pclass','sex','age','embarked']]
y= titanic['survived']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,train_size=0.7,random_state=0)

variables_numeric=titanic[['pclass','age']]
variables_categoric=titanic[['sex','embarked']]
#print(variables_numeric,variables_categoric)

pipeline_numeric=make_pipeline(SimpleImputer(missing_values=np.nan,strategy='mean'))
#pipeline_numeric.fit_transform(variables_numeric)    ##creer une pipline pour les valeures numeriques et categorielles
pipeline_categoric=make_pipeline(OneHotEncoder())##ImputerScaler: permet d'enlever les valeures manquantes 
#pipeline_categorique.fit_transform(variables_categoric)

##rassmebler les deux pipline en un seul transformer
trans=make_column_transformer((pipeline_numeric,['pclass','age']),(pipeline_categoric,['sex','embarked']))
#print(x,y)
###creer la pipline final 
pip=make_pipeline(trans,LinearDiscriminantAnalysis())
print(pip.fit(xtrain,ytrain))
print("prediction 4=>",pip.predict(xtest))
print("score 5",pip.score(xtest,ytest))
################################### pipeline_selector#############

variables_numeric_2=make_column_selector(dtype_include=np.number) ## selectione tout les variables de type nombre 
variables_categoric_2=make_column_selector(dtype_exclude=np.number) ## execlure toute les variables de type nombre 

pipeline_numeric_2=make_pipeline(StandardScaler())
pipeline_categoric_2=make_pipeline(OneHotEncoder())

##transformer principal 
trans_2=make_column_transformer((pipeline_numeric_2,variables_numeric_2),(pipeline_categoric_2,variables_categoric_2))

##pipline principal
pip_2=make_pipeline(trans_2,LinearDiscriminantAnalysis())
pip_2.fit(xtrain,ytrain)
print("")
print("prediction 5=>",pip_2.predict(xtest))
print("score 5 => ",pip_2.score(xtest,ytest))




   pclass  survived                                             name     sex  \
0       1         1                    Allen, Miss. Elisabeth Walton  female   
1       1         1                   Allison, Master. Hudson Trevor    male   
2       1         0                     Allison, Miss. Helen Loraine  female   
3       1         0             Allison, Mr. Hudson Joshua Creighton    male   
4       1         0  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  female   

       age  sibsp  parch  ticket      fare    cabin embarked  
0  29.0000      0      0   24160  211.3375       B5        S  
1   0.9167      1      2  113781  151.5500  C22 C26        S  
2   2.0000      1      2  113781  151.5500  C22 C26        S  
3  30.0000      1      2  113781  151.5500  C22 C26        S  
4  25.0000      1      2  113781  151.5500  C22 C26        S  
Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                       